# Streaming Body Object 

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='55SqeK2HiET2j44sS7eYJRzjuDPRtERLCYgJIn8x61aV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'digitalnaturalist-donotdelete-pr-mkoperlowqhvtb'
object_key = 'data.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


# Unzip The Zipped files

In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [9]:
pwd

'/home/wsuser/work'

# Importing the required libraries

In [11]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Load The Train And Test Datasets

In [12]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/home/wsuser/work/data/Training",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/home/wsuser/work/data/Testing",target_size=(224,224),batch_size=100)

Found 2916 images belonging to 6 classes.
Found 1562 images belonging to 6 classes.


# Model Building Using CNN

In [13]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

87910968/87910968 [==============================] - 2s 0us/step


#Freezing the weights in the pretrained model

In [14]:
for layer in base_model.layers:
  layer.trainable = False

#Flatening and adding dense layer


In [15]:
X = Flatten()(base_model.output)

#using sigmoid function as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

# Compiling the model

In [17]:
model = Model(base_model.input,X)

model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

# Model Checkpoint and Early Stopping

In [18]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./DN.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

# Model fitting

In [23]:
# Fitting the model 
model_new = model.fit(traindata, steps_per_epoch=len(traindata),epochs = 30,callbacks=call_back,validation_data=testdata)

Epoch 1/30
30/30 [==============================] - ETA: 0s - loss: 0.4688 - accuracy: 0.9554
Epoch 1: accuracy improved from 0.90089 to 0.95542, saving model to ./DN.h5
30/30 [==============================] - 143s 5s/step - loss: 0.4688 - accuracy: 0.9554 - val_loss: 0.1824 - val_accuracy: 0.9718
Epoch 2/30
30/30 [==============================] - ETA: 0s - loss: 0.2765 - accuracy: 0.9698
Epoch 2: accuracy improved from 0.95542 to 0.96982, saving model to ./DN.h5
30/30 [==============================] - 144s 5s/step - loss: 0.2765 - accuracy: 0.9698 - val_loss: 0.2090 - val_accuracy: 0.9859
Epoch 2: early stopping


# Saving The Model

In [24]:
model.save("DN.h5")

# Converting into tar file

In [25]:
!tar -zccvf Digital_Naturalist_model.tgz DN.h5

DN.h5


In [27]:
ls -1

data/
Digital_Naturalist_model.tgz
DN.h5


In [28]:
 !pip install watson_machine_learning_client --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.9/538.9 kB 24.3 MB/s eta 0:00:00


In [29]:
from ibm_watson_machine_learning import APIClient


wml_credentials={
    "url":"https://eu-gb.ml.cloud.ibm.com",
    "apikey":"MzaTpJooBFTc_sw9CwLzovQ945LfY3meAlIxBph4dHVP"
}

client=APIClient(wml_credentials)
client

In [34]:
def guid_from_space_name(client,space_name):
    space = client.spaces.getdetails()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [37]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------------------  ------------------------
ID                                    NAME                       CREATED
cc6201d6-4bf5-4994-8e76-524145007013  Digital_Naturalist_WebApp  2022-11-22T15:54:07.481Z
03d01618-4b6f-4eaa-9b5e-8a2727347c70  Digital_Naturalist         2022-11-20T13:24:45.319Z
------------------------------------  -------------------------  ------------------------


In [39]:
space_uid="cc6201d6-4bf5-4994-8e76-524145007013"
space_uid

'cc6201d6-4bf5-4994-8e76-524145007013'

In [41]:
client.set.default_space(space_uid)

'SUCCESS'

In [42]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [44]:
software_space_Uid =client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_Uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [46]:
model_details = client.repository.store_model(model='Digital_Naturalist_model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"Digital_Naturalist_Model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_Uid
    })

In [ ]:
model_id=client.respository.get

In [ ]:
model_id=client.repository.download(model_id, 'Digital_Naturalist_model.tgz')
